In [ ]:
# Creating a new ABM using a network structure

In [230]:
# loading packages
import random
import numpy as np
import networkx as nx
import pandas as pd
from collections import Counter

import mesa
from mesa import Agent, Model
from mesa.space import MultiGrid
from mesa.time import RandomActivation, BaseScheduler
from mesa.visualization.modules import CanvasGrid
from mesa.visualization.ModularVisualization import ModularServer
from mesa.datacollection import DataCollector

# CPS workflow

## World 


## Agents
Universal parameters:
problem_solving
communication

role_correct
role_distribution
role_negotiation lvl3
role_monitoring


Level 1

Level 2

Level 3
role_distributed=1

Level 4
Role_effect=1





## Teams are formed

## Problems
A team is posed with a problem with Task_complexity (1:3) and Size (1000)


Then we distribute role: 1 to len(team) (Do you fulfill your role or not?)
- each is assigned a "correct" role
- Then roles are more probable to become correct based on lvl2-4

- If Lvl 3/4 then roles are distributed correctly/evenly at first
- Self.Role=X

First we try to generate common-ground:
- Only lvl 2-4
- Based on agents Communication parameter
- Common_ground = 0 or 1

Then we simulate role_alignment:
- There some 0.8 prob that role stays correct
- Theres some 0.2 prob that role might shift to incorrect

Then Solutions are generated:
- generate some 50 to 150 solution (based on problem_solving)
- If lvl doesnt match task lvl then solution cannot be given

If some role > 1:
- Then conflict appears

If common_ground=0
- Lowest solution is chosen

If all roles are evenly distributed and common ground = 1:
    Then choose the highest solution

Adjustement of error/Meta negotiations/Self and team monitoring:
- Resolve_conflict
    - 50% prob of conflict resolvement
    - 90% if lvl 3 is present
- Reach_common_ground
    - 


In [238]:
class TeamAgent(Agent):
    """ An agent with X."""
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.cps_level = random.randint(1, 4)
        self.problem_solving = random.uniform(0, 1)
        self.team_members = []
        self.team_id = None
        
        self.current_role = 0
        self.correct_role = 0
        self.is_role_correct = 0
    

    def reset_team(self):
        self.team_members = []
        self.team_id = None


    # Joining teams
    def join_teams3(self):
        self.team_id = self.model.agent_team_dict.get(self.unique_id)

        #other_agent_ids = [agent_id for agent_id, t_id in self.model.agent_team_dict.items() if t_id == self.team_id and agent_id != self.unique_id]
        #self.team_members = other_agent_ids + [self.unique_id]
        members = [agent_id for agent_id, t_id in self.model.agent_team_dict.items() if t_id == self.team_id]
        self.team_members = members

        print("agent_id:", self.unique_id, "Team_id:", self.team_id, "Team_members:", self.team_members)

    # Creating a correct role
    def create_roles(self):

        x = list( range(1, len(self.model.teams[self.team_id])) )
        self.model.teams[self.team_id].append()
    

        self.model.teams[self.team_id]

    def get_correct_role(self):
        
        for i in self.team_members:
            self.correct_role = self.team_members.index(self.unique_id)

        print("Your correct role is:", self.correct_role)
    
    def get_current_role(self):

        


    def problem(self):
        pass

    def get_team(self):
        return self.team_members
        
    def step(self):
        self.reset_team()
        self.join_teams3()
        self.get_correct_role()

    

        
        
        

        
       

In [239]:
# Creating the model
class TeamModel(Model):
    def __init__(self, num_agents):
        self.num_agents = num_agents
        self.schedule = BaseScheduler(self)
        
        self.teams = {}
        self.agent_team_dict = {}
        self.groups = []

        self.problem_complexity = 0
        
        
        for i in range(self.num_agents):
            agent = TeamAgent(i, self)
            self.schedule.add(agent)

    # Creating team lists
    def create_teams(self):
        # Creating random teams of 2-5 agents based on the number of agents
        numbers = []
        x = 1
        while len(numbers) < num_agents:
            repetitions = random.randint(2, 5)
            numbers.extend([x] * repetitions)
            x += 1

        numbers = numbers[:num_agents]  # Trim the list to the desired number of agents
            
        random.shuffle(numbers) 

        self.groups = numbers

        # Creating the roles such that
        # each team has 2 to 5 roles specified by 1:n

        all_agents = list( range(0, self.num_agents) )
        self.teams = {
            "agent_id": all_agents,
            "team_id": self.groups
        }

        self.agent_team_dict = {agent_id: team_id for agent_id, team_id in zip(self.teams['agent_id'], self.teams['team_id'])}

    
        

    
    # creating a problem which is then used for solving
    def generate_problem(self): 
        self.problem_complexity = random.int(1,3)

    def get_agent_by_id(self, agent_id):
        return self.schedule._agents[agent_id]  # Accessing agent using its unique_id
    
    # A function which checks
    def collect_teams(self):
        print(self.teams)
        print(self.agent_team_dict)

    def reset_variables(self):
        self.teams = {}
        self.agent_team_dict = {}
        self.groups = []
        self.problem_complexity = 0

    def step(self):
        self.reset_variables()
        self.create_teams()
        self.schedule.step()
        self.collect_teams()




In [240]:

model = TeamModel(num_agents=20)

for i in range(2):
    model.step()

agent_id: 0 Team_id: 1 Team_members: [0, 1, 2, 10, 13]
Your correct role is: 0
agent_id: 1 Team_id: 1 Team_members: [0, 1, 2, 10, 13]
Your correct role is: 1
agent_id: 2 Team_id: 1 Team_members: [0, 1, 2, 10, 13]
Your correct role is: 2
agent_id: 3 Team_id: 6 Team_members: [3, 12]
Your correct role is: 0
agent_id: 4 Team_id: 5 Team_members: [4, 6, 8, 16, 19]
Your correct role is: 0
agent_id: 5 Team_id: 3 Team_members: [5, 9, 17]
Your correct role is: 0
agent_id: 6 Team_id: 5 Team_members: [4, 6, 8, 16, 19]
Your correct role is: 1
agent_id: 7 Team_id: 2 Team_members: [7, 15, 18]
Your correct role is: 0
agent_id: 8 Team_id: 5 Team_members: [4, 6, 8, 16, 19]
Your correct role is: 2
agent_id: 9 Team_id: 3 Team_members: [5, 9, 17]
Your correct role is: 1
agent_id: 10 Team_id: 1 Team_members: [0, 1, 2, 10, 13]
Your correct role is: 3
agent_id: 11 Team_id: 4 Team_members: [11, 14]
Your correct role is: 0
agent_id: 12 Team_id: 6 Team_members: [3, 12]
Your correct role is: 1
agent_id: 13 Team_i